In [1]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD
from keras.utils import to_categorical
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os

Using TensorFlow backend.


In [2]:
# initialize the data and labels
data = []
labels = []

In [3]:
# grab the image paths and randomly shuffle them
imagePaths = sorted(list(paths.list_images('animals')))
random.seed(42)
random.shuffle(imagePaths)

In [4]:
# loop over the input images
for imagePath in imagePaths:
    # load the image, resize the image to be 32x32 pixels (ignoring
    # aspect ratio), flatten the image into 32x32x3=3072 pixel image
    # into a list, and store the image in the data list
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (32, 32)).flatten()
    data.append(image)

    # extract the class label from the image path and update the
    # labels list
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)

In [5]:
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [6]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)

In [7]:
# convert the labels from integers to vectors (for 2-class, binary
# classification you should use Keras' to_categorical function
# instead as the scikit-learn's LabelBinarizer will not return a
# vector)
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [8]:
# define the 3072-1024-512-3 architecture using Keras
model = Sequential()
model.add(Dense(1024, input_shape=(3072,), activation="sigmoid"))
model.add(Dense(512, activation="sigmoid"))
model.add(Dense(len(lb.classes_), activation="softmax"))

In [9]:
# initialize our initial learning rate and # of epochs to train for
INIT_LR = 0.01
EPOCHS = 75

# compile the model using SGD as our optimizer and categorical
# cross-entropy loss (you'll want to use binary_crossentropy
# for 2-class classification)
print("[INFO] training network...")
opt = SGD(lr=INIT_LR)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

[INFO] training network...


In [10]:
lb.classes_

array(['cats', 'dogs', 'panda'], dtype='<U5')

In [11]:
# train the neural network
H = model.fit(trainX, trainY, validation_data=(testX, testY), epochs=EPOCHS, batch_size=32)

Train on 2250 samples, validate on 750 samples
Epoch 1/75
2250/2250 [==============================] - 2s 948us/step - loss: 1.1071 - accuracy: 0.3422 - val_loss: 1.0793 - val_accuracy: 0.3733
Epoch 2/75
2250/2250 [==============================] - 2s 978us/step - loss: 1.0849 - accuracy: 0.3960 - val_loss: 1.0983 - val_accuracy: 0.3147
Epoch 3/75
2250/2250 [==============================] - 2s 971us/step - loss: 1.0689 - accuracy: 0.4173 - val_loss: 1.0457 - val_accuracy: 0.4653
Epoch 4/75
2250/2250 [==============================] - 2s 962us/step - loss: 1.0513 - accuracy: 0.4276 - val_loss: 1.0831 - val_accuracy: 0.3160
Epoch 5/75
2250/2250 [==============================] - 2s 963us/step - loss: 1.0364 - accuracy: 0.4649 - val_loss: 1.0896 - val_accuracy: 0.3627
Epoch 6/75
2250/2250 [==============================] - 2s 965us/step - loss: 1.0237 - accuracy: 0.4956 - val_loss: 1.0263 - val_accuracy: 0.4827
Epoch 7/75
2250/2250 [==============================] - 2s 969us/step - loss:

2250/2250 [==============================] - 2s 827us/step - loss: 0.7988 - accuracy: 0.5951 - val_loss: 0.8972 - val_accuracy: 0.5547
Epoch 57/75
2250/2250 [==============================] - 2s 858us/step - loss: 0.7982 - accuracy: 0.5964 - val_loss: 0.8237 - val_accuracy: 0.6053
Epoch 58/75
2250/2250 [==============================] - 2s 827us/step - loss: 0.7967 - accuracy: 0.5893 - val_loss: 0.9118 - val_accuracy: 0.5213
Epoch 59/75
2250/2250 [==============================] - 2s 830us/step - loss: 0.7963 - accuracy: 0.5880 - val_loss: 0.8262 - val_accuracy: 0.5787
Epoch 60/75
2250/2250 [==============================] - 2s 819us/step - loss: 0.7926 - accuracy: 0.5987 - val_loss: 0.8620 - val_accuracy: 0.5933
Epoch 61/75
2250/2250 [==============================] - 2s 819us/step - loss: 0.7879 - accuracy: 0.6182 - val_loss: 0.8845 - val_accuracy: 0.5800
Epoch 62/75
2250/2250 [==============================] - 2s 918us/step - loss: 0.7918 - accuracy: 0.6018 - val_loss: 0.8618 - val_

In [12]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=lb.classes_))

[INFO] evaluating network...
              precision    recall  f1-score   support

        cats       0.68      0.19      0.30       236
        dogs       0.45      0.67      0.54       236
       panda       0.70      0.83      0.76       278

    accuracy                           0.58       750
   macro avg       0.61      0.57      0.53       750
weighted avg       0.61      0.58      0.55       750



In [13]:
# plot the training loss and accuracy
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["accuracy"], label="train_accuracy")
plt.plot(N, H.history["val_accuracy"], label="val_accuracy")
plt.title("Training Loss and Accuracy (Simple NN)")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig("output/plot.png")

In [14]:
# save the model and label binarizer to disk
print("[INFO] serializing network and label binarizer...")
model.save("saved_models/simple_nn.model")
f = open("output/simple_nn_lb.pickle", "wb")
f.write(pickle.dumps(lb))
f.close()

[INFO] serializing network and label binarizer...
